In [91]:
# imports
import cv2 as cv
import numpy as np
import pandas as pd
import os


In [94]:
def findPainting(image):
#     Get edges and make them wider
    edges = cv.Canny(gray, 33, 70)
    dilated = cv.dilate(edges, np.ones((3,3), np.uint8))
    
#     Get all contours of the edges
    contours, hierarchy = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv.contourArea, reverse=True)[:10]
    
#     Check for every contour if it is a polygon with 4 corners
    for c in contours:
        app = cv.approxPolyDP(c, 0.05*cv.arcLength(c, True), True)
        if len(app) == 4:
            break
    app = app.reshape(4,2)
    
#     Get the transformation matrix and execute the transformation on the image to obtain a translated painting
    width = np.max(app[:,0]) - np.min(app[:,0])
    height = np.max(app[:,1]) - np.min(app[:,1])
    M = cv.getPerspectiveTransform(np.float32(app), np.float32([[0,0],[0,height],[width,height],[width,0]]))
    result = cv.warpPerspective(image,M,(width, height),flags=cv.INTER_LINEAR)
    
    return result

In [95]:
image_path = "./Database/Computervisie 2020 Project Database/dataset_pictures_msk"
db_log = pd.DataFrame("./Database_log.csv")

for row in db_log:
    image = cv.imread(image_path+'/'+row[0]+'/'+row[1]+".png")
    image_copy = image.copy()
    cv.imshow("titel eerst", image)
    cv.waitKey()
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    result = findPainting(image)
    cv.imshow("titel tweed", result)
    cv.waitKey()

cv.destroyAllWindows()